<a href="https://colab.research.google.com/github/ankush170/Cousera-test.github.io/blob/master/densenet_model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip,RandomRotation,RandomZoom,RandomContrast
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
import math
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications.densenet import preprocess_input

In [3]:
batch_size = 32
img_height = 224
img_width = 224

epochs=30

import os
import pathlib
data_dir = '/content/drive/MyDrive/dataset_unsorted'
data_dir= pathlib.Path(data_dir)

In [4]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

17180


In [5]:
train_ds = tf.keras.utils.image_dataset_from_directory(data_dir,
                                                       shuffle = True,
                                                       label_mode= "int",
                                                       color_mode= 'rgb',
                                                       labels= 'inferred',
                                                       validation_split = 0.2,
                                                       subset = 'training',
                                                       seed = 123,
                                                       image_size=(img_height, img_width),
                                                       batch_size=batch_size)

Found 17180 files belonging to 31 classes.
Using 13744 files for training.


In [6]:
val_ds = tf.keras.utils.image_dataset_from_directory(data_dir,
                                                     shuffle= True,
                                                     validation_split=0.2,
                                                     label_mode= "int",
                                                     color_mode= 'rgb',
                                                     labels= 'inferred',
                                                     subset="validation",
                                                     seed=123,
                                                     image_size=(img_height, img_width),
                                                     batch_size=batch_size)

Found 17180 files belonging to 31 classes.
Using 3436 files for validation.


In [7]:
class_names = train_ds.class_names
print(class_names)

['Chilli - Fusorium Wilt', 'Chilli - Helicoverpa', 'Chilli - N Def', 'Chilli - Thrips', 'Groundnut - Alternaria', 'Groundnut - Healthy', 'Groundnut - LEAF HOPPER AND JASSIDS', 'Groundnut - Leaf Miner', 'Groundnut - Rust', 'Groundnut - Tobaco Caterpillaer', 'Groundnut - late and early leaf spot', 'Maize - Aphids', 'Maize - Ear Worm', 'Maize - Eye Spot', 'Maize - Healthy_V5', 'Maize - Magnesium', 'Maize - Nitrogen Deficiancy', 'Maize - Potassium', 'Maize - Southern Leaf Blight', 'Paddy - BLB', 'Paddy - Brown spot', 'Paddy - Grain discolaration', 'Paddy - Grasshoppers', 'Paddy - Leaf roller', 'Paddy - Phosphorous', 'Paddy - Potassium deficincy', 'Paddy - Rice hispa', 'Paddy - Stemborer', 'Paddy - Zinc', 'Sorghum - Anthracnose', 'Sorghum - Fall Army Worm']


In [9]:
data_augmentation =Sequential( [
    RandomFlip("horizontal_and_vertical",
                input_shape=(img_height,
                             img_width,
                             3)),
                RandomRotation(0.2),
                RandomContrast(0.1),
                RandomZoom(0.2),
                               ])

In [10]:
val_batches = tf.data.experimental.cardinality(val_ds)
test_dataset = val_ds.take(val_batches // 5)
val_ds = val_ds.skip(val_batches // 5)
print('Number of validation batches: %d' % tf.data.experimental.cardinality(val_ds))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

Number of validation batches: 87
Number of test batches: 21


In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().shuffle(1000).prefetch(buffer_size =AUTOTUNE)


normalization_layer=layers.Rescaling(1./255)

In [ ]:
normalized_ds= train_ds.map(lambda x,y: (normalization_layer(x),(y)))
image_batch,labels_batch = next(iter(normalized_ds))
first_image= image_batch[0]
print(np.min(first_image),np.max(first_image))

In [ ]:
num_classes = len(class_names)
base_densenet121_model = Sequential([
                                 data_augmentation,
                        layers.Rescaling( scale=1./255),
                        layers.Resizing(img_height, img_width),

                        DenseNet121(input_shape=(img_height, img_width,3), weights="imagenet", include_top=False)
] , name = 'base_densenet121_model')

base_densenet121_model.trainable = False
base_densenet121_model.summary()


custom_densenet121_model = Sequential([
        base_densenet121_model,
        GlobalAveragePooling2D(),
                                                   
        Dense(512,activation='relu'),
        Dropout(0.2),
        Dense(num_classes , activation="softmax")
        ], name = "custom_densenet121_model")

custom_densenet121_model.summary()

In [ ]:
custom_densenet121_model.compile(optimizer=Adam(learning_rate=0.001*3),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
                         metrics=['accuracy']
                         )

In [ ]:
early = EarlyStopping(monitor="val_loss", 
                      patience=math.floor(epochs*0.1))

learning_rate_reduction = ReduceLROnPlateau(monitor="val_loss",
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.3,
                                            min_lr=0.000001)
# saves the model in the current directory as the best validation accuracy is achieved.
modelcheck = ModelCheckpoint('best_model(DenseNet121).hdf5', monitor='val_accuracy',verbose=1,save_best_only=True,mode='max')

In [ ]:
history = custom_densenet121_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[modelcheck, learning_rate_reduction, early],
  verbose=1
)